In [1]:
import pandas
from bs4 import BeautifulSoup
import splinter
import re

In [2]:
monsters_to_import = [
    'http://dndroll.wikidot.com/creatures:giant-poisonous-snake',
    'http://dndroll.wikidot.com/creatures:ape',
    'http://dndroll.wikidot.com/creatures:spider'
]
database = 'monsters_wikidot.csv'

In [23]:
def get_AC(line):
    return re.match(r'[^0-9]*([0-9]+)', line).group(1)
def get_HP(line):
    return re.match(r'[^0-9]*(.*)', line).group(1)
def get_stat(row):
    tds = row.find_all("td")
    return f'{tds[1].text} ({tds[2].text})'
def abbreviate(string):
    s2 = string.replace("Skills ", "").replace("Senses ","").replace("Languages ","").replace("Challenge", "CR").replace("—","").replace("\n\n","\n").replace("\n","<br>").replace(" ft.","'")
    s2 = re.sub(r'Passive Perception [0-9]*<br>', "", s2)
    return s2

def get_monster(url):
    monster = {}
    monster['url'] = url
    browser = splinter.Browser("firefox")
    browser.visit(url)
    soup = BeautifulSoup(browser.html, 'html.parser')
    browser.quit()
    main_block = soup.find("div", id="main-content")
    name = main_block.find("div", id="page-title").text.strip()
    stats_table = main_block.find("table")
    page_block = main_block.find("div", id="page-content")
    if page_block.find("p").findNext("table") == None: # no paragraph before stats table
        for line in page_block.text.split("\n"):  
            line = line.strip()
            if line == "":
                continue
            if 'meta' in monster.keys():
                monster['meta'] = line
                print(f"setting meta to {meta}")
            if re.match(r'.*Armor\s+Class', line):
                monster['AC'] = get_AC(line)
            elif re.match(r'.*Hit\s+Points', line):
                monster['HP'] = get_HP(line)
            elif re.match(r'.*ft\.', line) or re.match(r'.*feet', line):
                monster['movement'] = line
            elif re.match(r'.*Ability', line):
                break;
    else:
        first_paragraph = main_block.find("div", id="page-content").find("p")
        fp_lines = first_paragraph.text.split("\n")
        monster['meta'] = fp_lines[0]
        monster['AC'] = get_AC(fp_lines[1])
        monster['HP'] = get_HP(fp_lines[2])
        monster['movement'] = fp_lines[3]
        
    rows= stats_table.find_all("tr")
    stats = [get_stat(row) for row in rows[1:7]]
    monster['str'] = stats[0]
    monster['dex'] = stats[1]
    monster['con'] = stats[2]
    monster['int'] = stats[3]
    monster['wis'] = stats[4]
    monster['cha'] = stats[5]
    second_paragraph = stats_table.findNext("p")
    tidbits = abbreviate(second_paragraph.text)
    monster['tidbits'] = abbreviate(second_paragraph.text)
    
    third_paragraph = second_paragraph.findNext("p")
    monster['attributes'] = third_paragraph.text.replace(" ft.","'")    
    return monster

In [24]:
m = get_monster(monsters_to_import[2])
m

{'url': 'http://dndroll.wikidot.com/creatures:spider',
 'AC': '12',
 'HP': '1 (1d4 - 1)',
 'movement': 'Speed 20 ft., climb 20 ft.',
 'str': '2 (-4)',
 'dex': '14 (+2)',
 'con': '8 (-1)',
 'int': '1 (-5)',
 'wis': '10 (+0)',
 'cha': '2 (-4)',
 'tidbits': "Stealth +4<br>Darkvision 30', CR 0 (10 XP)",
 'attributes': 'Spider Climb. The spider can climb difficult surfaces, including upside down on ceilings, without needing to make an ability check.'}